<a href="https://colab.research.google.com/github/molabokchi/bokchi_open_lab/blob/main/deepdriver_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. deepdriver experiment 및 run 생성

In [1]:
pip install deepdriver==0.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 KB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 3.3 MB/s eta 0:00:00
  Created wheel for assertpy: filename=assertpy-1.1-py3-none-any.whl size=42917 sha256=a7ed824a11afec42a062a331481db9a4735f8ed450cc3a179acca3ec33052367
  Stored in directory: /root/.cache/pip/wheels/57/86/c9/1310be6ddfb540daa0bf1ac204526837aa0a8b0e79f32855ff
Successfully built assertpy
  Attemptin

In [2]:
import deepdriver

In [3]:
deepdriver.setting(http_host="54.180.86.146:9011" ,grpc_host="54.180.86.146:19051")

In [4]:
deepdriver.login(key="ZmIyNWQxNGJkMzUxYTVjODQ2NjM5NTgzOTM0YTM2OGE2ZmJiY2M2MWMwOWQ0OWFkNjU2YzNkM2UxMjA0YTVkZg==")

True

In [5]:
def make_exp_name():
  import socket
  from datetime import datetime
  host_name = socket.gethostname()
  date = datetime.today().strftime("%Y%m%d") 

  return "exp" + "_" +"torch_"+host_name +"_"+date

In [6]:
# experiment init & config hyperparam
deepdriver.init(exp_name= make_exp_name(), 
                config={ 'learning_rate':0.001,'context_size':2 , 'embedding_dim':10})


DeepDriver initialized
Team Name=molamola.python
Exp Name=exp_torch_7ec061fdbb57_20230202
Run Name=run-1
Run URL=http://54.180.86.146:9111/experi/molamola.python/exp_torch_7ec061fdbb57_20230202/run-1/run/chart


#5. example

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # noqa: N812
import torch.optim as optim

In [ ]:
#test example
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [
    ([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
    for i in range(len(test_sentence) - 2)
]
_hook_handles ={}



vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

has_cuda = torch.cuda.is_available()

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), deepdriver.config.embedding_dim, deepdriver.config.context_size)
model = model.cuda() if has_cuda else model
optimizer = optim.SGD(model.parameters(), lr=deepdriver.config.learning_rate)
deepdriver.watch(model, log_freq=1000)

for i in range(100):
    print("epoch :" + str(i))
    total_loss = 0
    for batch_i, (context, target) in enumerate(trigrams):

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor(
            [word_to_ix[w] for w in context], dtype=torch.long
        )
        context_idxs = context_idxs.cuda() if has_cuda else context_idxs

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        target = torch.tensor([word_to_ix[target]], dtype=torch.long)
        target = target.cuda() if has_cuda else target
        loss = loss_function(log_probs, target)

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
        if batch_i%10 ==0:
          deepdriver.log({"batch_loss": loss.item()})
    losses.append(total_loss)
print(losses)  # The loss decreased ev

True
True
True
True
True
epoch :0
epoch :1
epoch :2
epoch :3
epoch :4
epoch :5
epoch :6
epoch :7
epoch :8
Uploading: [./deepdriver/run/5063/chart/gradients/embeddings.weight.CHART.json] |██████████████████████████████| [100.0%] [1/1]epoch :9
epoch :10
epoch :11
epoch :12
epoch :13
epoch :14
epoch :15
epoch :16
epoch :17
Uploading: [./deepdriver/run/5063/chart/gradients/embeddings.weight.CHART.json] |██████████████████████████████| [100.0%] [1/1]epoch :18
epoch :19
epoch :20
epoch :21
epoch :22
epoch :23
epoch :24
epoch :25
epoch :26
Uploading: [./deepdriver/run/5063/chart/gradients/embeddings.weight.CHART.json] |██████████████████████████████| [100.0%] [1/1]epoch :27
epoch :28
epoch :29
epoch :30


In [ ]:
deepdriver.finish()